#### Making Class Instances Behave Like Functions Using __call__():

- In Python, classes can be designed such that their instances can be called like regular functions. This is achieved by implementing the special method __call__() in the class. When you call an instance of the class, Python internally calls the __call__() method, passing any arguments given to the instance as if it were a function.

- In LangChain, the __call__() method can be particularly useful when you're working with chains of functions or objects, allowing you to compose objects or operations seamlessly. When dealing with chains in LangChain, you'll often need to call objects (or components) in a sequence, making the __call__() method a useful tool for implementing behavior that resembles function calls.

In [ ]:
# Function __call__() , Example for making the instance of a class behave like a function
class Adder:
    def __init__(self, number):
        self.number = number

    def __call__(self, other_number):
        return self.number + other_number

# Create an instance of Adder
add_five = Adder(5) # intialize the  instance variable with constructor
# Now you can call it like a function
result = add_five(3)  # This will return 8 , Here , Indirectly calling __call__(),  here self have 5 value and other will have 3 value equivalent to 
## add_five.number =self.number=5, also other=3
print(result)  # Output: 8

### How It Works:

- The `Adder` class has a constructor `__init__()` that initializes the instance with a number (`self.number`).
- The `__call__()` method is defined in the `Adder` class. This method takes an argument `other_number`, which will be the number you pass when you call the object (like a function).
- In the `add_five(3)` call, Python internally calls `add_five.__call__(3)`, where:
  - `self.number` is `5` (from `Adder(5)`).
  - `other_number` is `3` (the argument passed during the function call).
- The result is `5 + 3 = 8`.


In [ ]:
class Runnable:
    def __init__(self, func):
        self.func = func

    # Operator overloading
    def __or__(self, other):
        print('or')
        def chained_func(*args, **kwargs):
            # this is nested function in which we create chain of funtion
            #here the other function will consume output on this first function
            #upon which we call the or operator first element
            print("Inside the chained function")
            return other(self.func(*args, **kwargs))
        print('chained func end')
        return Runnable(chained_func)
    # The __call__() function implementation , make an instance of a class behave like a function
    def __call__(self, *args, **kwargs):
        return self.func(*args, **kwargs)

In [ ]:
#Let's implement this to take the value 3, add 5

def double(x):
  return 2 * x

def add_one(x):
  return x + 1

# wrap the functions with Runnable
runnnable_double = Runnable(double) # Here we are using the object of Runnable class and assigning the reference of double function into instance variable func ie  runnable_duble.func= double reference
runnable_add_one = Runnable(add_one) # Here we are using the object of Runnable class and assigning the reference of add function into instance variable func ie  runnable_add_one.func= add_one reference 
                                     # runnable_add_one.func= add_one reference

chain = runnnable_double.__or__(runnable_add_one)   # it will return the refernce of the internal function chained_func [Which is the internal function] and this call for runnnable_double
#  consider the  other(self.func(*args, **kwargs)) --->>   self.func will replaced with double as this call belong to the object runnnable_double .
print('chain type:', type(chain))
chain(5)  # should return 11
print(chain(5))
#chain the runnable functions together
double_then_add_one =  runnnable_double |  runnable_add_one  
print('double_then_add_one:', type(double_then_add_one))
#invoke the chainLCEL
result = double_then_add_one(5) 
print(result) 


### Three Runnable Instances:
- **`runnnable_double`**: wraps the `double` function.
- **`runnable_add_one`**: wraps the `add_one` function.
- **`chain`**: wraps the `chained_func`, which combines `runnnable_double` and `runnable_add_one`.
- The __or__ method is used to overload the pipe (|) operator to chain functions.
- The __call__ method allows instances of Runnable to be called like functions.

---

### What Happens When `chain(5)` is Called?

When you call `chain(5)`, this triggers the `__call__` method of the `Runnable` class. Here's the flow:

1. `chain` wraps the `chained_func`, so `chain(5)` calls `chained_func(5)`.

---

### Inside `chained_func`:

The following code snippet is executed:

```python
def chained_func(*args, **kwargs):
    return other(self.func(*args, **kwargs))
```

- **`self`** refers to the original object that invoked the `__or__` method, i.e., `runnnable_double`.
- **`self.func`** is the `double` function (because `runnnable_double.func = double`).
- So, `self.func(5)` calls `double(5)`, returning `10`.

---

### Why Does `self` Refer to `runnnable_double`?

This happens because `chained_func` was defined inside the `__or__` method of `runnnable_double`. When `chained_func` runs, it "remembers" the context of `self` where it was created. In Python, this is called a **closure**.

1. **`chained_func` "closes over" `self`**, which was `runnnable_double` at the time of its creation.
2. This is why, even though `chain` is a new `Runnable` object, the `self.func` inside `chained_func` still refers to the `func` attribute of `runnnable_double`.

---

### The Result of `double(5)`:

The result of `double(5)` (i.e., `10`) is passed to `other`:

- **`other`** is `runnable_add_one`, which wraps the `add_one` function.
- `other(10)` calls `runnable_add_one.__call__(10)`, which executes `add_one(10)` and returns `11`.


Three Runnable Instances:

    - runnnable_double: wraps double function.
    - runnable_add_one: wraps add_one function.
    - chain: wraps the chained_func, which combines runnnable_double and runnable_add_one.

What Happens When chain(5) is Called?
    - When you call chain(5), this triggers the __call__ method of the Runnable class. Here's the flow:

    - chain wraps the chained_func, so chain(5) calls chained_func(5).

Inside chained_func: the folowing code snippets
    - def chained_func(*args, **kwargs):
        return other(self.func(*args, **kwargs))

    - self refers to the original object that invoked the __or__ method, i.e., runnnable_double.
      self.func is the double function (because runnnable_double.func = double).
      So, self.func(5) calls double(5), returning 10.

The result of double(5) (i.e., 10) is passed to other:

    - other is runnable_add_one, which wraps the add_one function.
      other(10) calls runnable_add_one.__call__(10), which executes add_one(10) and returns 11.

Why Does self Refer to runnnable_double?

    - his happens because chained_func was defined inside the __or__ method of runnnable_double. When chained_func runs, it "remembers" the context of self where it was created. In Python, this is called a closure.

    - chained_func "closes over" self, which was runnnable_double at the time of its creation.
      This is why, even though chain is a new Runnable object, the self.func inside chained_func still refers to the func attribute of runnnable_double.



In [1]:
class CustomBool:
    def __init__(self, value):
        self.value = value

    def __or__(self, other):
        # Simulate the 'or' operator
        return CustomBool(self.value or other.value)

    def __repr__(self):
        return f"CustomBool({self.value})"

# Example usage
a = CustomBool(True)
b = CustomBool(False)
c = CustomBool(True)

result1 = a | b  # Should simulate 'True or False'
result2 = b | b  # Should simulate 'False or False'
result3 = a | c  # Should simulate 'True or True'

print(result1)  # Output: CustomBool(True)
print(result2)  # Output: CustomBool(False)
print(result3)  # Output: CustomBool(True)